In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 현재 디렉터리 확인
!pwd

/content


In [3]:
# import해서 불러올 폴더 디렉터리로 변경 -> 즉, 폴더의 위치
# 반드시, 파일들을 같은 폴더에 해야해요!

# %cd '폴더 저장 경로'

%cd /content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/D-attn

/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/D-attn


In [4]:
# 변경한 디렉터리의 파일 확인

!ls

 config.ipynb   main.ipynb    model.py	   'Upload_ver02_D-attn model.ipynb'   utils.py
 config.py      model.ipynb   __pycache__   utils.ipynb


In [5]:
# 변경한 디렉토리로 잘 되었는지 확인
!pwd

/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/D-attn


In [6]:
!pip install torch torchvision


# main

In [7]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import pickle
import numpy as np
from sklearn import metrics



import pandas as pd

from torch.utils.data import Dataset, DataLoader
from torch.utils.data.dataloader import default_collate

In [8]:
from config import Config
from utils import get_loader, ReviewDataset, my_collate
from model import CNNDLGA, LocalAttention, GlobalAttention

In [9]:

from torch.optim import Adam



def to_var(x):
    """CUDA 변환 함수"""
    if torch.cuda.is_available():
        x = x.cuda()
    return torch.autograd.Variable(x)


def evaluate(model, dataloader, criterion, config):
    """Validation 또는 Test 데이터를 이용한 모델 평가 함수"""
    model.eval()
    total_loss = 0
    with torch.no_grad():
        for user_review, item_review, labels in dataloader:
            user_review = to_var(user_review)
            item_review = to_var(item_review)
            labels = to_var(labels)

            outputs = model(user_review, item_review)
            loss = criterion(outputs, labels)
            total_loss += loss.item() * labels.size(0)
    return total_loss / len(dataloader.dataset)


def main():
    config = Config()

    # 데이터 로더 생성
    train_loader = get_loader(config.train_file, config.emb_file, batch_size=config.batch_size, shuffle=True)
    valid_loader = get_loader(config.valid_file, config.emb_file, batch_size=config.batch_size, shuffle=False)
    test_loader = get_loader(config.test_file, config.emb_file, batch_size=config.batch_size, shuffle=False)

    print(f"train/valid/test: {len(train_loader)}/{len(valid_loader)}/{len(test_loader)}")
    print("=" * 80)

    # 모델 초기화
    model = CNNDLGA(
        input_size=config.input_size,
        embed_size=config.embed_size,
        win_size=config.win_size,
        channels_local=config.channels_local,
        channels_global=config.channels_global,
        hidden_size=config.hidden_size,
        output_size=config.output_size
    ).to(config.device)

    # 손실 함수와 최적화기 설정
    criterion = torch.nn.MSELoss()
    optimizer = Adam(model.parameters(), lr=config.learning_rate)

    # 학습
    best_valid_loss = float('inf')
    for epoch in range(config.num_epochs):
        model.train()
        total_loss = 0

        for i, (user_review, item_review, labels) in enumerate(train_loader):
            user_review = to_var(user_review)
            item_review = to_var(item_review)
            labels = to_var(labels)

            # Forward
            outputs = model(user_review, item_review)
            loss = criterion(outputs, labels)

            # Backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            # 출력
            if (i + 1) % 10 == 0:
                print(f"Epoch [{epoch+1}/{config.num_epochs}], Step [{i+1}/{len(train_loader)}], "
                      f"Loss: {loss.item():.4f}")

        print(f"Epoch [{epoch+1}] Training Loss: {total_loss / len(train_loader):.4f}")

        # Validation
        valid_loss = evaluate(model, valid_loader, criterion, config)
        print(f"Epoch [{epoch+1}] Validation Loss: {valid_loss:.4f}")

        # Best Model 저장
        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), f"{config.save_path}best_model.pkl")
            print(f"Best model saved at Epoch [{epoch+1}] with Validation Loss: {best_valid_loss:.4f}")

    # 테스트
    print("Testing on the Test Dataset...")
    model.load_state_dict(torch.load(f"{config.save_path}best_model.pkl"))
    test_loss = evaluate(model, test_loader, criterion, config)
    print(f"Test Loss: {test_loss:.4f}")

    # 결과 저장
    with open(f"{config.save_path}test_results.pkl", "wb") as f:
        pickle.dump({'test_loss': test_loss}, f)
    print("Test results saved.")

if __name__ == "__main__":
    main()


train/valid/test: 400/50/50


/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/D-attn/utils.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(user_review, dtype=torch.float), \
/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/D-attn/utils.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(user_review, dtype=torch.float), \


Epoch [1/10], Step [10/400], Loss: 16.3819
Epoch [1/10], Step [20/400], Loss: 17.2872
Epoch [1/10], Step [30/400], Loss: 16.3283
Epoch [1/10], Step [40/400], Loss: 13.6879
Epoch [1/10], Step [50/400], Loss: 15.2212
Epoch [1/10], Step [60/400], Loss: 13.2503
Epoch [1/10], Step [70/400], Loss: 9.4509
Epoch [1/10], Step [80/400], Loss: 8.0317
Epoch [1/10], Step [90/400], Loss: 6.1076
Epoch [1/10], Step [100/400], Loss: 2.4685
Epoch [1/10], Step [110/400], Loss: 3.6662
Epoch [1/10], Step [120/400], Loss: 2.5739
Epoch [1/10], Step [130/400], Loss: 2.2157
Epoch [1/10], Step [140/400], Loss: 1.9059
Epoch [1/10], Step [150/400], Loss: 2.5056
Epoch [1/10], Step [160/400], Loss: 2.1417
Epoch [1/10], Step [170/400], Loss: 1.9893
Epoch [1/10], Step [180/400], Loss: 3.6546
Epoch [1/10], Step [190/400], Loss: 2.7776
Epoch [1/10], Step [200/400], Loss: 2.3329
Epoch [1/10], Step [210/400], Loss: 2.3367
Epoch [1/10], Step [220/400], Loss: 2.5318
Epoch [1/10], Step [230/400], Loss: 2.1793
Epoch [1/10], 

/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/D-attn/utils.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(user_review, dtype=torch.float), \
/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/D-attn/utils.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(user_review, dtype=torch.float), \


Epoch [1] Validation Loss: 2.4616
Best model saved at Epoch [1] with Validation Loss: 2.4616


...

Epoch [10/10], Step [10/400], Loss: 2.8771
Epoch [10/10], Step [20/400], Loss: 2.3845
Epoch [10/10], Step [30/400], Loss: 2.7503
Epoch [10/10], Step [40/400], Loss: 2.0039
Epoch [10/10], Step [50/400], Loss: 2.5741
Epoch [10/10], Step [60/400], Loss: 2.1982
Epoch [10/10], Step [70/400], Loss: 3.0026
Epoch [10/10], Step [80/400], Loss: 2.8174
Epoch [10/10], Step [90/400], Loss: 2.8216
Epoch [10/10], Step [100/400], Loss: 2.5673
Epoch [10/10], Step [110/400], Loss: 2.8654
Epoch [10/10], Step [120/400], Loss: 2.0488
Epoch [10/10], Step [130/400], Loss: 3.9383
Epoch [10/10], Step [140/400], Loss: 3.3639
Epoch [10/10], Step [150/400], Loss: 2.1721
Epoch [10/10], Step [160/400], Loss: 2.8752
Epoch [10/10], Step [170/400], Loss: 2.9863
Epoch [10/10], Step [180/400], Loss: 2.4072
Epoch [10/10], Step [190/400], Loss: 2.4254
Epoch [10/10], Step [200/400], Loss: 2.9969
Epoch [10/10], Step [210/400], Loss: 2.8958
Epoch [10/10], Step [220/400], Loss: 2.3417
Epoch [10/10], Step [230/400], Loss: 2.36

/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/D-attn/utils.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(user_review, dtype=torch.float), \
/content/drive/MyDrive/IDS/amaxon reviews 2023/최종본/import 방식/D-attn/utils.py:63: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:278.)
  return torch.tensor(user_review, dtype=torch.float), \


Epoch [10] Validation Loss: 2.4578
Testing on the Test Dataset...


<ipython-input-9-0727fbabad6a>:95: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(f"{config.save_path}best_model.pkl"))
/content/drive/MyDriv

Test Loss: 2.4163
Test results saved.
